In [1]:
from data_utils import *

In [2]:
import json
import os
import pickle
from pathlib import Path
from PIL import Image
import sys

## Sample an experimental configuration

Uses the `data_utils.SyntheticEC` class to randomly sample the semantic features and blindspots for a single EC.

In [3]:
ec = SyntheticEC()
ec.sample(verbose = True)

[('background', 'presence', 1), ('background', 'relative-position', 1), ('circle', 'color', 0), ('circle', 'presence', 1), ('circle', 'texture', 1), ('square', 'number', 0), ('square', 'presence', 1), ('square', 'texture', 1)]
[('background', 'presence', 1), ('background', 'relative-position', 0), ('circle', 'color', 0), ('circle', 'presence', 1), ('square', 'number', 1), ('square', 'presence', 1), ('square', 'size', 0), ('square', 'texture', 0)]

Features
background ['texture', 'relative-position']
square ['size', 'texture', 'number']
circle ['color', 'texture']

Blindspots
{'background': {'relative-position': 1}, 'circle': {'color': 0, 'presence': 1, 'texture': 1}, 'square': {'number': 0, 'texture': 1}}
{'background': {'relative-position': 0}, 'circle': {'color': 0, 'presence': 1}, 'square': {'number': 1, 'size': 0, 'texture': 0}}



In [4]:
ec_dir = './data/sample_ec'

ec.save_dataset(directory = ec_dir,
               num_train_images_per_bucket = 20,
               num_val_images_per_bucket = 20,
               num_test_images_per_bucket = 20)

Generating data in:  ./data/sample_ec/test
Generating data in:  ./data/sample_ec/val
Generating data in:  ./data/sample_ec/train


## Model Training: Demo

Fine-tunes a pretrained ResNet-18 using the sample EC.

In [5]:
from torchvision.models import resnet18
import torch
from torch_utils import *

In [6]:
# Load the model
model = resnet18(pretrained = True)

# Change the classification layer
model.fc = torch.nn.Linear(in_features = 512, out_features = 1)

In [7]:
files, labels = load_phase(ec_dir, 'train')
dataset = ImageDataset(files, labels)
train_loader = get_loader(dataset, batch_size = 16)

# change to 'cuda' if available
device = 'cpu'

In [8]:
from tqdm.notebook import tqdm as tqdm

In [9]:
epochs = 10
lr = 1e-4

optimizer = torch.optim.Adam(model.parameters(), lr = lr)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
losses = []

for epoch in tqdm(range(epochs)):
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        # forward pass
        y_hat = model(x)
        loss = loss_fn(y_hat, y)
        
        # step
        loss.backward()
        optimizer.step()
        
    losses.append(loss.item())

  0%|          | 0/10 [00:00<?, ?it/s]